In [1]:
import pandas as pd
import numpy as np
from kaggle_handler import handler

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.tree import DecisionTreeClassifier

# Downloading/Loading Dataset

In [3]:
Assets = handler('yasserh/titanic-dataset')

Directory 'Assets' already exists.
Datasets already exist in Assets folder
['Titanic-Dataset.csv']
 Change Add_more parameter to download more datasets


In [4]:
Assets

['Titanic-Dataset.csv']

In [5]:
df = pd.read_csv('Assets/Titanic-Dataset.csv',
                usecols=['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch','Fare', 'Embarked'])
df.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [6]:
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),
                                                 df['Survived'],
                                                 test_size=.2)
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((712, 7), (712,), (179, 7), (179,))

In [7]:
X_train.isna().sum()

Pclass        0
Sex           0
Age         142
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

# SimpleImputation
- Columns applied [Age, Embarked]
- Age Imputed with mean
- Embarked Imputed with Most Frequent Value

In [8]:
si_age = SimpleImputer()
si_emabarked = SimpleImputer(strategy='most_frequent')

X_train_age = si_age.fit_transform(X_train[['Age']])
X_train_embarked = si_emabarked.fit_transform(X_train[['Embarked']])

X_test_age = si_age.transform(X_test[['Age']])
X_test_embarked = si_emabarked.transform(X_test[['Embarked']])

X_train_age.shape, X_train_embarked.shape

((712, 1), (712, 1))

In [9]:
X_train['Age'] = np.reshape(X_train_age, (X_train_age.shape[0],))
X_test['Age'] = np.reshape(X_test_age, (X_test_age.shape[0],))

# OneHotEncoding
- Columns applied [Sex, Embarked]

In [10]:
ohe_sex = OneHotEncoder(sparse_output=False,
                        handle_unknown='ignore',
                        dtype=np.int32)
ohe_embarked = OneHotEncoder(sparse_output=False,
                        handle_unknown='ignore',
                        dtype=np.int32)

X_train_sex = ohe_sex.fit_transform(X_train[['Sex']])
X_train_embarked = ohe_embarked.fit_transform(X_train_embarked)

X_test_sex = ohe_sex.transform(X_test[['Sex']])
X_test_embarked = ohe_embarked.transform(X_test_embarked)

X_train_sex.shape, X_train_embarked.shape

((712, 2), (712, 3))

# MinMaxScaling
- Columns Scaled [Age, Fare]

In [11]:
scale = MinMaxScaler()

X_train_age_fare = scale.fit_transform(X_train[['Age','Fare']])
X_test_age_fare = scale.transform(X_test[['Age','Fare']])

X_train_age_fare.shape

(712, 2)

# Combaning All Transformed Columns

In [12]:
X_train_rem = X_train.drop(columns=['Age', 'Fare', 'Embarked', 'Sex'])
X_test_rem = X_test.drop(columns=['Age', 'Fare', 'Embarked', 'Sex'])

X_train_rem.head(2)

,Pclass,SibSp,Parch
270,1,0,0
176,3,3,1


In [13]:
X_train_transform = np.concat((X_train_rem,X_train_age_fare,X_train_sex,X_train_embarked),axis=1)
X_test_transform = np.concat((X_test_rem,X_test_age_fare,X_test_sex,X_test_embarked),axis=1)

X_train_transform.shape

(712, 10)

# Taning Model Using DecisionTree

In [14]:
clf = DecisionTreeClassifier()
clf.fit(X_train_transform,y_train)

DecisionTreeClassifier()

In [15]:
y_pred = clf.predict(X_test_transform)
y_pred

array([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1,
       1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1,
       1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0])

In [16]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true=y_test, y_pred=y_pred)

0.7821229050279329

# Exporting/Saving Model

In [17]:
import pickle
import os

if 'without_pipline_models' not in os.listdir():
    os.mkdir('without_pipline_models')
pickle.dump(ohe_sex, open('without_pipline_models/ohe_sex.pkl','wb'))
pickle.dump(ohe_embarked, open('without_pipline_models/ohe_embarked.pkl','wb'))
pickle.dump(scale, open('without_pipline_models/age_fare_scale.pkl','wb'))
pickle.dump(clf, open('without_pipline_models/clf.pkl','wb'))